In [11]:
myName = "Jing Li"
name_split = myName.split(',')
print(name_split)

['Jing Li']


In [11]:
from datetime import datetime
currTime = datetime.now()
print(currTime)

2020-04-18 17:51:15.382171


In [12]:
myStr = ['alpha','beta','gamma']
myStr[:-1]

['alpha', 'beta']

In [130]:
import bibtexparser
import datetime

def RepresentsInt(s):
    try:
        int(s)
        return True
    except ValueError:
        return False


# check if the date entry is the correct format, i.e., year-month-day
def IsDateFormatValid(str):
    try:
        datetime.datetime.strptime(str, '%Y-%m-%d')
        return True
    except ValueError as err:
        print(err)
        return False

# check if the date entry is the correct format, i.e., year-month
def IsOtherDateFormatValid(str):
    try:
        datetime.datetime.strptime(str, '%Y-%m')
        return True
    except ValueError as err:
        print(err)
        return False
    

# remove all decorative expression in a string
def supetrim(string):
    return string.replace("\\" , "").replace("{" , "").replace("}" , "").replace("\n"," ")


# convert Jan, Feb, Mar etc to 01, 02, 03, etc.
def month_string_to_number(string):
    m = {
        'jan':1,
        'feb':2,
        'mar':3,
        'apr':4,
        'may':5,
        'jun':6,
        'jul':7,
        'aug':8,
        'sep':9,
        'oct':10,
        'nov':11,
        'dec':12
        }
    s = string.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')

        
# convert 01, 02, 03 etc to Jan, Feb, Mar etc
def month_number_to_string(num):
    m = {
        1:'Jan',
        2:'Feb',
        3:'Mar',
        4:'Apr',
        5:'May',
        6:'Jun',
        7:'Jul',
        8:'Aug',
        9:'Sep',
        10:'Oct',
        11:'Nov',
        12:'Dec'
        }

    try:
        out = m[num]
        return out
    except:
        raise ValueError('Not a month')

    
pubtype_dict = {
    'Uncategorized': '"0"',
    'misc': '"0"',
    'inproceedings': '"1"',
    'conference': '"1"',
    'article': '"2"',
    'submitted': '"3"',
    'preprint': '"3"',
    'techreport': '"4"',
    'book': '"5"',
    'incollection': '"6"',
    'phdthesis': '"7"',
    'mastersthesis': '"7"',
    'patent': '"8"',
}

with open('../publications.bib') as bibtex_file:
    bibtex_str = bibtex_file.read()

bib_database = bibtexparser.loads(bibtex_str)
numRef = 0
currDate = datetime.datetime.now()  # today's date in year, month, day.

for entry in bib_database.entries:
    numRef += 1
    print(str(numRef) + ": " + entry['ID'])
    
    # formatting publication date to be YYYY-MM-DD
    if 'date' in entry:
        print('original date entry:', entry['date'])
        # date format: year-month-day
        if IsDateFormatValid(entry['date']):
            origDate = datetime.datetime.strptime(entry['date'], '%Y-%m-%d')  #in datetime.datetime datetype
            pubDate = str(origDate.year) + '-' + str(origDate.month).zfill(2) + '-' + str(origDate.day).zfill(2)
            print('formatted date with existing date (year-month-day): ' + pubDate)
        # date format: year-month
        elif IsOtherDateFormatValid(entry['date']):
            origDate = datetime.datetime.strptime(entry['date'], '%Y-%m')
            pubDate = str(origDate.year) + '-' + str(origDate.month).zfill(2) + '-01'
            print('formatted date with existing date (year-month): ' + pubDate)
        pubMonth = month_number_to_string(origDate.month) # used for "publication" item in index.md
        print('publication month: ' + pubMonth)
    elif 'year' in entry:
        pubDate = entry['year']
        print('original date entry:', pubDate)
        if 'month' in entry:
            print('publication month:', entry['month'])
            # month is in digital format, i.e. 01, 02, ...,12
            if RepresentsInt(entry['month']):
                month = entry['month'] # string
            else:
                # convert Jan, Feb, etc to 01, 02, ...
                month = str(month_string_to_number(entry['month']))
            pubDate = pubDate + '-' + month.zfill(2) + '-01'
            pubMonth = month_number_to_string(int(month))
            print('formatted date with existing year and month entry: ', pubDate)
            print('publication month: '+ pubMonth)
        else: 
            # first, check if the status of the publication
            if 'pubstate' in entry:
                pubState = entry['pubstate']
                print(pubState)
                if pubState == 'forthcoming' or pubState == 'upcoming' or pubState == 'accepted':
                    pubDate = str(currDate.year) + '-' + str(currDate.month) + '-' + str(currDate.day)
                    print('upcoming publication date: ', pubDate)
                else:  #draft, submitted, under review, preprint, etc.
                    pubDate = pubDate + '-01-01'                
            else: # most likely the entry is for an old publication with only published year               
                pubDate = pubDate + '-01-01'
            print('publication with only year entry: ', pubDate)
    else:  # rare case, no year, month, day info
        print('Error: No publication date - check the bib entry')
        
        
    # Treating the publication journal, conference, etc.
    publication = 'publication: "_'

    ## for proceedings, it should display title, _booktitle_, ser.series, location, month, year.
    if 'booktitle' in entry:
        publication = publication + supetrim(entry['booktitle']) + '_'
        ## including short name for conference proceedings
        if 'series' in entry:
            publication = publication + ', ser. ' + supetrim(entry['series'])
        if 'location' in entry:
                publication = publication + ', ' + supetrim(entry['location'])
        if 'pubMonth' in locals():  # if month is in 'date' entry or 'month' entry
                publication = publication + ', ' + supetrim(pubMonth)
        if 'year' in entry:
                publication = publication + ', ' + supetrim(entry['year'])
        print(publication + '"\n')

    ## for articles, it should display title, _journal_, **volumne**, (number), page_start-page_end
    elif 'journal' in entry:
        publication = publication + supetrim(entry['journal']) + '_'
        if 'volume' in entry:
            publication = publication + ', **' + supetrim(entry['volume']) + '**'
        if 'number' in entry:
            publication = publication + ', (' + supetrim(entry['number']) + ')'
        if 'pages' in entry:
            publication = publication + ', ' + supetrim(entry['pages'])
        if 'year' in entry:
            publication = publication + ', ' + supetrim(entry['year'])

        print(publication + '"\n')
    
    print('\n')
#        if len(entry['date']) == 10:
#            print('date: ' + entry['date'])
#        else: 
#            print('date: ' + entry['date'] + '-01')
            
#    if 'ENTRYTYPE' in entry:
#        print('publication_types: ['+pubtype_dict[entry['ENTRYTYPE']]+']\n')
#    if 'abstract' in entry:
#        print('abstract: "'+entry['abstract'] +'"\n')


        

1: khoram2019toco
original date entry: 2019-12-18
formatted date with existing date (year-month-day): 2019-12-18
publication month: Dec


2: khoram2019interleaved
original date entry: 2019-12-18
formatted date with existing date (year-month-day): 2019-12-18
publication month: Dec


3: alex2019mlsys
original date entry: 2019
publication month: 2
formatted date with existing year and month entry:  2019-02-01
publication month: Feb


4: ratner2019sysml
original date entry: 2019-03-29
formatted date with existing date (year-month-day): 2019-03-29
publication month: Mar
publication: "_arXiv preprint arXiv:1904.03257_, **abs/1904.03257**, 2019"



5: zha2020jssc
original date entry: 2020-01-13
formatted date with existing date (year-month-day): 2020-01-13
publication month: Jan
publication: "_IEEE Journal of Solid-State Circuits (**JSSC**)_, **55**, (4), 908--919, 2020"



6: shukla2018frontiers
original date entry: 2018-02-14
formatted date with existing date (year-month-day): 2018-02-14
pu

In [108]:
import re
   
def removeEmptyString(inStr):
    while("" in inStr):
        inStr.remove("") 
    
str1 = 'meza2012:report:nvm'
if ':' in str1:
    str2 = str1.replace(':', "-")
print(str2)

string_to_split = 'li2009phd'
a = re.split('[^a-zA-Z]', string_to_split)
b = re.split('[a-zA-Z]', string_to_split)

#removeEmptyString(a)
#removeEmptyString(b)

#c = re.split('[^A-Z]', a[1])
#c = re.split('[A-Z][a-zA-Z]*', a[1])
#removeEmptyString(c)
#d = re.findall('[A-Z][a-zA-Z]*', a[1])
print(a)
print(b)
#print(c)
#print(d)

#outputStr = '-'.join([a[0]] + b + c + d)

#print(outputStr.lower())


meza2012-report-nvm
['li', '', '', '', 'phd']
['', '', '2009', '', '', '']


In [11]:
testStr = 'zhang2017fpgaCNN'
def splitnonalpha(s):
   pos = 1
   while pos < len(s) and s[pos].isalpha():
      pos+=1
   return (s[:pos], s[pos:])
splitnonalpha(testStr)

('zhang', '2017fpgaCNN')

In [99]:
import datetime

# theDate = datetime.datetime.strptime('2018-9-14', '%Y-%m-%d')
# print(theDate, theDate.day)

theDate = datetime.datetime.strptime('2018-9', '%Y-%m')
print(theDate, theDate.month)

#help(datetime.datetime)

# check if the date entry is the correct format, i.e., year-month-day
def IsDateFormatValid(str):
    try:
        dateEntry = datetime.datetime.strptime(str, '%Y-%m-%d')
        print(dateEntry)
        return True
    except ValueError as err:
        print(err)
        return False

# check if the date entry is the correct format, i.e., year-month
def IsOtherDateFormatValid(str):
    try:
        dateEntry = datetime.datetime.strptime(str, '%Y-%m')
        print(dateEntry)
        return True
    except ValueError as err:
        print(err)
        return False

#IsDateFormatValid('2018-11')
#IsOtherDateFormatValid('2018-11')
currTime = datetime.datetime.now()
# print('Current Time:', currTime.year, currTime.month, currTime.day)
currTime.year == 2020
newDate = currTime


2018-09-01 00:00:00 9


In [95]:
'01'.zfill(2)

'01'

In [129]:
# convert 01, 02, 03 etc to Jan, Feb, Mar etc
def month_number_to_string(num):
    m = {
        1:'Jan',
        2:'Feb',
        3:'Mar',
        4:'Apr',
        5:'May',
        6:'Jun',
        7:'Jul',
        8:'Aug',
        9:'Sep',
        10:'Oct',
        11:'Nov',
        12:'Dec'
        }

    try:
        out = m[num]
        return out
    except:
        raise ValueError('Not a month')
        
month = month_number_to_string(int('02'))
print(month)
if 'month' in locals():
    print('month:', month)

Feb
month: Feb
